In [137]:
import fastjsonschema
import jsonschema
import json
from types import FunctionType
from typing import Callable, Set, Dict, Any, Union, List, Tuple
from inspect import getmembers, isfunction
from warnings import warn
from uuid import uuid4, UUID
from functools import partial


def getAttrDictIfExists(obj, attr):
    if attr in obj:
        return {attr: getattr(obj, attr)}
    return {}


def stringWrapper(func: Callable) -> Callable:
    def wrapped(*argv, **kwargs):
        return str( func(*argv, **kwargs) )
    
    return wrapped


class JSONObject(object):
    '''JSONObject holds all the information about an object and can be completely described by a
    JSON schema. It can be serialized to JSON and deserialized in Python. Special attributes that
    include functionality are the object id, the object class, and references to other objects.
    
    The id and class attributes can be specified by functions that automatically generate their
    values. By default, the id is autogenerated and the class is taken from the Python object. 
    When deserializing an object and the idKey and classKey are not set, then the class will 
    automatically call the generators.
    
    References are handled separately on serialization and de-serialization. On serialization,
    refMarkup is added to references of other JSONObjects. On de-serialization, the graph container
    object calls the refGenerator function to build a set of refKeys using information
    '''
    
    idKey: str = "" # The key under which the id will be saved. Set to empty to avoid generating
    idGenerator: Callable = staticmethod( lambda: str(uuid4()) ) # The function used to generate the id (ie be the value of idKey).
    classKey: str = "" # The key under which the class definition will be stored. Set to empty to ignore
    classGenerator: Callable = lambda self: [t.__name__ for t in self.__class__.mro() if t.__name__ != "object"] # Returns
    schema: Dict = dict()
    saveSchema: bool = False # Saves schema as part of object
    __funcs__: Dict = dict()
    refKeys: Set = set()
    refGenerator: Callable = lambda self: [k for k,v in self.__dict__.items() if v.get(self.refProperty)] # Returns true if the refProperty markup is found. Can be something else that returns a bool.
    refProperty: str = "graphRef" # Gets added to refMarkup as the way that references are indicated
    refMarkup: List[Callable] = [
        lambda obj: getAttrDictIfExists(obj, obj.idKey),
        lambda obj: getAttrDictIfExists(obj, obj.classKey)
    ]
    setDefault: bool = False #TODO
    removeEmpty: bool = True # TODO
    
    
    # Initialize a blank function that later gets overwritten by the fastjsonschema validator
    @staticmethod
    def validate(data):
        return None
    
    
    # The method for handling non-default types. Overwrites the default method of json library.
    @staticmethod
    def JSONDefault(obj):
        if isinstance(obj, UUID):
            return str( obj )
        if isinstance(obj, JSONObject):
            return {"@id": str( getattr(obj, '@id') )}
                
        return json.JSONEncoder.default(obj)
    
    
    # Sets the validator and JSONDefault for the class upon creation of a new instance.
    # Updates some Callables and Dictionaries based on keys defined as class attributes.
    def __new__(cls, **kwargs):
        # Set the markup to include the refProperty
        cls.refMarkup += [lambda obj: {cls.refProperty: True}]
        
        cls.validate = staticmethod( fastjsonschema.compile(cls.schema) )
        
        def JSONDefault(obj):
            if isinstance(obj, UUID):
                return str( obj )
            if isinstance(obj, JSONObject):
                rep = dict()
                
                if obj.idKey:
                    rep.update({cls.idKey: str( getattr(obj, obj.idKey) )})
                    
                if obj.classKey:
                    rep.update({cls.classKey: str( getattr(obj, obj.classKey) )})
                    
                for c in cls.refMarkup:
                    if isfunction(c):
                        rep.update( c(obj) )
                    else:
                        rep.update( v )
                        
                return rep

            return json.JSONEncoder.default(obj)
        
        cls.JSONDefault = staticmethod( JSONDefault )
        
        return super().__new__(cls)
    
    
    def __init__(self, **kwargs):
        # Init an id
        if self.idKey and (self.idKey not in kwargs):
            setattr(self, self.idKey, self.idGenerator())
        # Set the class
        if self.classKey and (self.classKey not in kwargs):
            setattr(self, self.classKey, self.classGenerator())
            
        # Init required vals
        ## Issue if "required" keys override keys checked in setattr
        if "required" in self.schema:
            initial_vals = {k: kwargs[k] for k in self.schema["required"]}

            self.validate(initial_vals)
            self.__dict__.update(initial_vals)
        
        # Iterate to enforce checks on keys
        for k, v in kwargs.items():
            setattr(self, k, v)
            
            
    def json(self):
        return json.dumps(self.__dict__, indent=2, default=self.JSONDefault)
    
    
    def toMapping(self):
        return json.loads(self.json())
        
        
    def __str__(self):
        return self.json()
    
    
    def __setattr__(self, key, val):
        if key == "schema":
            raise KeyError("Cannot Reset Schema")
            
        if isfunction(val):
            if key not in self.__dict__:
                warn("Functions will not be serialized. Add a callable JSONObject instead.")
            else:
                raise ValueError(f"{key} is already defined as an attribute and cannot be " \
                                 f"redefined as a function")
        
        
        self.validate( {**self.__dict__, key: val} )
        
        self.__dict__.update( {key: val} )
        
        
    def __getattr__(self, key):
        if key in self.__funcs__:
            return self.__funcs__[key]
        
        try:
            return self.__dict__[key]
        finally:
            raise AttributeError(f"{key} not a member of {self}")
            
            
    def __contains__(self, key):
        if key in self.__dict__:
            return True
        if key in self.__funcs__:
            return True
        return False
    
    
class JSONGraph:
    bannedId: Callable = lambda x: False
    nodes: Dict[str, JSONObject] = dict()
    edges: Dict[Tuple[str, str, str], Tuple[JSONObject, JSONObject, str]] = dict() # (src, tgt, edgeType)
    
    
    def addObject(self, obj: JSONObject) -> None:
        assert isinstance(obj, JSONObject), "JSONGraph only supports objects of type JSONObject"
        
        self.addNode(obj)
        self.addObjectsRefs(obj, obj.__dict__)
        
        
    def addNode(self, obj: JSONObject):
        assert obj.idKey, "Cannot add object without key."
        
        self.nodes[getattr(obj, obj.idKey)] = obj
        
        
    def addEdge(self, obj1: JSONObject, obj2: JSONObject, edgeType:str=""):
        assert obj1.idKey, "Cannot add object without key."
        assert obj2.idKey, "Cannot add object without key."
        
        self.edges[(getattr(obj1, obj1.idKey), getattr(obj2, obj2.idKey), edgeType)] = (obj1, obj2, edgeType)
        
        
    def addObjectsRefs(self, obj: JSONObject, d: Dict) -> None:
        for k, v in d.items():
            if isinstance(v, JSONObject):
                self.addEdge(obj, v, k)
                
                if getattr(v, v.idKey) not in self.nodes:
                    self.addNode(v)
                    self.addObjectsRefs(v, v.__dict__)
                
            elif isinstance(v, dict):
                addObjectsRefs(obj, v)
                
                
    def toMapping(self) -> Dict:
        mapping = dict()
        
        for k in self.edges.keys():
            if k in mapping:
                mapping[k[0]].append((k[2], k[1]))
            else:
                mapping[k[0]] = [(k[2], k[1])]
                
        return mapping
    
                
    def __str__(self) -> str:
        return json.dumps(self.toMapping(), indent=2)
                
            
        
        
        
    
    
joe = JSONObject(hello="world")
bob = JSONObject(friend=joe)
joe.friend = bob

print(joe)
print(bob)

jg = JSONGraph()

try:
    jg.addObject(joe)
except AssertionError:
    print("successfully required a key")

{
  "hello": "world",
  "friend": {
    "graphRef": true
  }
}
{
  "friend": {
    "graphRef": true
  }
}
successfully required a key


In [138]:
class schemaOrg(JSONObject):
    idKey = "@id"
    classKey = "@type"
    
joe = schemaOrg(hello="world")
bob = schemaOrg(friend=joe)
joe.friend = bob

print(joe)
print(bob)

jg.addObject(joe)

print(jg)

{
  "@id": "274d31ad-52b3-41c4-904d-5d22ce6c67c8",
  "@type": [
    "schemaOrg",
    "JSONObject"
  ],
  "hello": "world",
  "friend": {
    "@id": "b9422180-1290-4db7-9d99-ba647b47aff3",
    "@type": [
      "schemaOrg",
      "JSONObject"
    ],
    "graphRef": true
  }
}
{
  "@id": "b9422180-1290-4db7-9d99-ba647b47aff3",
  "@type": [
    "schemaOrg",
    "JSONObject"
  ],
  "friend": {
    "@id": "274d31ad-52b3-41c4-904d-5d22ce6c67c8",
    "@type": [
      "schemaOrg",
      "JSONObject"
    ],
    "graphRef": true
  }
}
{
  "274d31ad-52b3-41c4-904d-5d22ce6c67c8": [
    [
      "friend",
      "b9422180-1290-4db7-9d99-ba647b47aff3"
    ]
  ],
  "b9422180-1290-4db7-9d99-ba647b47aff3": [
    [
      "friend",
      "274d31ad-52b3-41c4-904d-5d22ce6c67c8"
    ]
  ]
}


In [101]:
print( schemaOrg(**joe.toMapping()) )

{
  "@id": "097e2bb6-4595-4364-afcf-91b387709165",
  "@type": [
    "schemaOrg",
    "JSONObject"
  ],
  "hello": "world",
  "friend": {
    "@id": "3f2afd0d-4fc6-40ea-9aac-c4c145eb6929",
    "@type": [
      "schemaOrg",
      "JSONObject"
    ],
    "graphRef": true
  }
}


In [108]:
class foo:
    pass

a = foo()
b = foo()

a.next = b

a.next == b

True

In [109]:
json.dumps({"material": a})

TypeError: Object of type foo is not JSON serializable

In [102]:
class Specific(JSONObject):
    schema = {"type": "object", "properties": {"thing": {"type": "string"}}}
    
spec = Specific(thing="5")

print(spec)
print(spec.thing)

spec.other = 5

print(spec)

try:
    spec.schema = {}
    spec.thing = 5
except (KeyError, fastjsonschema.JsonSchemaValueException):
    print("int value successfully caught")
    
print(spec)
print(spec.schema)

{
  "thing": "5"
}
5
{
  "thing": "5",
  "other": 5
}
int value successfully caught
{
  "thing": "5",
  "other": 5
}
{'type': 'object', 'properties': {'thing': {'type': 'string'}}}


In [ ]:
repository.types.Material({"name": "inconel"})

repository.sync()

In [103]:
class Specific(JSONObject):
    schema = {"type": "object", "properties": {"thing": {"type": "string"}}}
    
spec = Specific(thing=foo)

print(spec)
print(spec.thing)

spec.other = 5

print(spec)

try:
    spec.schema = {}
    spec.thing = 5
except (KeyError, fastjsonschema.JsonSchemaValueException):
    print("int value successfully caught")
    
print(spec)
print(spec.schema)

<ipython-input-98-4f40a0ff0da1>:149: UserWarning: Functions will not be serialized. Add a callable JSONObject instead.
  warn("Functions will not be serialized. Add a callable JSONObject instead.")


JsonSchemaValueException: data.thing must be string